## CS431/631 Big Data Infrastructure
### Winter 2018 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Hao Dong
* **ID:** 20757585

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [1]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
from collections import defaultdict
from math import log
# f = open("Shakespeare.txt","r")

with open('Shakespeare.txt') as f:
    # use dictionary "tokens" and "pairs" 
    # for "tokens", use one-token query as keys, the number of line on which it occurs as value
    # for "pairs", use two-token queries as keys, the number of line on which it occurs as the 
    # first item of the list of value and PMI as the second item of the list of value
    tokens = {}
    pairs = defaultdict(list)
    line_num = 0;
    for line in f:
        # tokenize, one line at a time
        t_line = set(simple_tokenize(line))
        
#         if(t_line):
        line_num += 1
        
        # for every token in this line, if it exists in the dictionary "tokens" already, its value plus one;
        # otherwise, add it into the dictionary "tokens".
        for token in t_line:
            if(token in tokens):
                tokens[token] += 1
            else:
                tokens[token] = 1    
        
        # for every possible pair in this line, if it exists in the dictionary "pairs" already, its value
        # plus one; otherwise, add it into the dictionary "pairs" , the number of line on which it occurs 
        # as the first item of the list of value
        for t1 in t_line:
            for t2 in t_line:
                if t1 != t2:
                    if((t1,t2) in pairs):
                        pairs[(t1,t2)][0] += 1
                    else:
                        pairs[(t1,t2)].append(1)

# with "pairs" saved in the above cell, calculate every pairs' PMI and save it into "pairs" as 
# the second item of the list of value
for pair in pairs.keys():
    t1 = pair[0]
    t2 = pair[1]  
    PMI = log(pairs[pair][0] * line_num / (tokens[t1] * tokens[t2]),10)
    pairs[pair].append(PMI)

# print the length of two dictionaries repectively showing the number of distinct tokens and distinct token pairs
print(len(tokens))
print(len(pairs))



25975
1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [3]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10

import operator
from collections import OrderedDict



# use "tokens" and "pairs" above
while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
                

        # save pairs which have n(x) >= threhold into "largest_pairs"
        largest_pairs = defaultdict(list)
        for pair in pairs.keys():
            if (q_tokens[0] == pair[0] and pairs[pair][0] >= threshold):
                largest_pairs[pair].append(pairs[pair][0])
                largest_pairs[pair].append(pairs[pair][1])
        # sort the largest_pairs by PMI
        largest_pairs = OrderedDict(sorted(largest_pairs.items(), key=lambda e: e[1][1], reverse=True))

        if(q_tokens[0] in tokens.keys()):
            # print the number of lines on which q_tokens[0] occurs
            print("  n({0}) = {1}".format(q_tokens[0],tokens[q_tokens[0]]))

            # print the five  𝑦 's for which  𝑃𝑀𝐼(𝑥,𝑦)  is largest.
            print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
            i = 0
            for pair in largest_pairs.keys():
                print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".
                format(pair[0], pair[1],largest_pairs[pair][0],round(largest_pairs[pair][1],3))) 
                i += 1
                if(i == 5):
                    break
 
    elif len(q_tokens) == 2:
        if((q_tokens[0],q_tokens[1]) in pairs.keys()):
            print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],pairs[(q_tokens[0],q_tokens[1])][0]))
            print("  PMI({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],round(pairs[(q_tokens[0],q_tokens[1])][1],3)))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): the
Input a positive integer frequency threshold: 1000
  n(the) = 24300
  high PMI tokens with respect to the (threshold: 1000):
    n(the,of) = 7266,  PMI(the,of) = 0.343
    n(the,king) = 1117,  PMI(the,king) = 0.302
    n(the,is) = 2434,  PMI(the,is) = 0.146
    n(the,all) = 1050,  PMI(the,all) = 0.141
    n(the,in) = 2863,  PMI(the,in) = 0.133
Input 1 or 2 space-separated tokens (return to quit): you i
  n(you,i) = 2854
  PMI(you,i) = 0.186
Input 1 or 2 space-separated tokens (return to quit): the death
  n(the,death) = 209
  PMI(the,death) = 0.091
Input 1 or 2 space-separated tokens (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

*As the input file grows larger, say, 1000 times larger than original input, the execution of program will get much slower.*

*Assume the file has n lines, and  m tokens in total.
Then the time efficiency of my code is O($(\frac{m}{n})^2 n$),that is O($\frac{m^2}{n}$).*

*That is ,when input file contains much more tokens which means its size becomes much larger, the execution of program will get much slower. And it may eventually fail to run，since in my code has to save two dictionary into the memory. If the file is large enough, the memory may be not big enough, the code may crash.*


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.